<a href="https://colab.research.google.com/github/rubyspch/Colaboratory-Notes/blob/main/Fixing_null_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cleaning data.
What to do with data that is NOT null, there is data there, but it is an invalid response. 
A number of methods to do this.


In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({
    'Sex': ['M', 'F', 'F', 'D', '?'],
    'Age': [29, 30, 24, 290, 25],
})

Sex here is defined as m or f. 
Some values are not m or f.
You can find these with:

In [3]:
df['Sex'].unique() #this tells you each value given

array(['M', 'F', 'D', '?'], dtype=object)

In [4]:
df['Sex'].value_counts() #this gives you the different values and a count of each

F    2
?    1
D    1
M    1
Name: Sex, dtype: int64

Once you have found them you can fix the problem by replacing them:

In [5]:
df['Sex'].replace('D', 'F')

0    M
1    F
2    F
3    F
4    ?
Name: Sex, dtype: object

In [7]:
df['Sex'].replace({'D': 'F'}) #replace can be done with a dictionary, this is how you change multiple at once.

0    M
1    F
2    F
3    F
4    ?
Name: Sex, dtype: object

In [ ]:
df.replace({
    'Sex': {
        'D': 'F',
        'N': 'M'
    },
    'Age': {
        290: 29
    }
}) #this is the syntax for replacing data in more than once column at once

For the problem with age, one value is higher than human age can be, you need a more complex fix.

Here the 0 is a type and it should be 29. You can account for any instance that this error was made by dividing any age over 100 by 10:

In [ ]:
df.loc[df['Age'] > 100, 'Age'] = df.loc[df['Age'] > 100, 'Age'] / 10


Duplicates in Series:
Important methods `duplicated()` and `drop_duplicates()`

In [8]:
ambassadors = pd.Series([
    'France',
    'United Kingdom',
    'United Kingdom',
    'Italy',
    'Germany',
    'Germany',
    'Germany',
], index=[
    'Gérard Araud',
    'Kim Darroch',
    'Peter Westmacott',
    'Armando Varricchio',
    'Peter Wittig',
    'Peter Ammon',
    'Klaus Scharioth '
])

In [9]:
ambassadors.duplicated()


Gérard Araud          False
Kim Darroch           False
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig          False
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

working top-down, the code counts the duplicates. The first instance of a country does not count as duplicate, but subsequent ones do.

Adding attribute `keep=last` flips it, so the code keeps the last mention of a country.
Attribute `keep=False` makes dupes true and non dupes false

In [10]:
ambassadors.duplicated(keep=False)

Gérard Araud          False
Kim Darroch            True
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig           True
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

Drop:
gets rid of duplicated, doesn't return a Boolean

In [11]:
ambassadors.drop_duplicates() #gets rid of the duplicated cases

Gérard Araud                  France
Kim Darroch           United Kingdom
Armando Varricchio             Italy
Peter Wittig                 Germany
dtype: object

keep last and keep false work with drop- they do the same as duplicated but they drop the values instead of returning a Boolean

Duplicates with DataFrames

If 2 rows are exactly the same they are considered duplicates.

In [13]:
players = pd.DataFrame({
    'Name': [
        'Kobe Bryant',
        'LeBron James',
        'Kobe Bryant',
        'Carmelo Anthony',
        'Kobe Bryant',
    ],
    'Pos': [
        'SG',
        'SF',
        'SG',
        'SF',
        'SF'
    ]
})
players.duplicated()

0    False
1    False
2     True
3    False
4    False
dtype: bool

^Here the Kobe at [2] is a dupe of [0]. But the Kob at [4] is not, because the 'Pos' is different for that one.

Adding attribute subset = 'column-name' asks the code to check dupes within a specific column. e.g:

In [14]:
players.duplicated(subset=['Name'])

0    False
1    False
2     True
3    False
4     True
dtype: bool

^Now the Kobe at [4] is considered a dupe.

Keep last and false still can apply. And this can all apply to drop_duplicates() too.

**Text Handling**

Dealing with invalid text data using example df:

In [20]:
df = pd.DataFrame({
    'Data': [
        '1987_M_US _1',
        '1990?_M_UK_1',
        '1992_F_US_2',
        '1970?_M_   IT_1',
        '1985_F_I  T_2'
]})

Use info to find out the attribute of the data. category, string, datetime all have different methods that can be used with them. Str methods are similar to pure python methods (e.g. split, replace)

You can use split to separate the data, as the data follows a pattern where the answers are split by underscores. So:

In [21]:
df['Data'].str.split('_')

0       [1987, M, US , 1]
1       [1990?, M, UK, 1]
2        [1992, F, US, 2]
3    [1970?, M,    IT, 1]
4      [1985, F, I  T, 2]
Name: Data, dtype: object

In [22]:
df= df['Data'].str.split('_', expand=True) #expand means the splits create new columns. Assign it to df.

In [23]:
df.columns=['Year','Sex','Country','Children'] #add your own columns bc they are indexed by default when expanded

In [24]:
df

,Year,Sex,Country,Children
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [25]:
df['Year'].str.contains('\?') #use contains to create a Boolean mask for where the ? are

0    False
1     True
2    False
3     True
4    False
Name: Year, dtype: bool

In [27]:
df['Country'].str.strip() #use strip to remove whitespace OR replace:
df['Country'].str.replace(' ', '')

0    US
1    UK
2    US
3    IT
4    IT
Name: Country, dtype: object

In [ ]:
df['Year'].str.replace(r'(?P<year>\d{4})\?', lambda m: m.group('year'))

#can use regular expressions here to replace.